## Latent Dirichlet Allocation

In [ ]:
from octis.models.LDA import LDA
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.optimization.optimizer import Optimizer
from skopt.space.space import Integer
import numpy as np
import time
import matplotlib.pyplot as plt
import json
from wordcloud import WordCloud
from tqdm import tqdm


In [ ]:
NUM_ITERS = 15
NUM_OPTIMIZATION_CALLS = 15

In [ ]:
dataset = Dataset()
dataset.load_custom_dataset_from_folder('data/processed/dataset')

## Metrics

In [ ]:
NUM_ITERS = 15
NUM_OPTIMIZATION_CALLS = 15 # Bottleneck for computation time 10m calls approx 3 hours

In [ ]:
td, ch = TopicDiversity(topk=10), Coherence(texts = dataset.get_corpus(), topk= 10, measure = 'c_v') # Initialize metrics

## Without optimization

In [ ]:
# Create Model, our  arbitrary assumption on the initial number of topics: 10 
model = LDA(num_topics=5, alpha = 'auto', passes= 50, iterations = 200, update_every = 1, chunksize = 1000)

In [ ]:
coherence_list_no_opt, topic_diversity_list_no_opt = [], []

for i in tqdm(range(NUM_ITERS)):
  output = model.train_model(dataset)
  coherence_list_no_opt.append(ch_score := ch.score(output))
  topic_diversity_list_no_opt.append(td_score := td.score(output))

print("Mean coherence: ", np.mean(coherence_list_no_opt), "\nMean topic diversity: ", np.mean(topic_diversity_list_no_opt))

## With optimization

In [ ]:
model = LDA(alpha = 'auto', passes= 50, iterations = 200, update_every = 1, chunksize = 1000)

In [ ]:
# Optimization for hyperparameters based on coherence

search_space = {"num_topics": Integer(low=5, high=35)}
optimizer=Optimizer()


In [ ]:

start = time.time() # Just to see, to be removed.

# This uses the default optimization method (Bayesian optimization) and the default metric (coherence) to optimize the model, try also random search.

optResult = optimizer.optimize(model, dataset, ch, search_space, save_path="results/test_LDA", # path to store the results
                            number_of_call = NUM_OPTIMIZATION_CALLS, # number of optimization iterations: rule of thumb 15*num_hyperparameters but takes a lot of time so 30 instead
                            model_runs = 5, # number of runs of the topic model: can be increased but takes more time
                            plot_best_seen = True, # plot the best seen value of the metric
                            extra_metrics = [td], # track also the topic diversity
                            plot_model = True, # plot the topic model
                            early_step = 10, # number of iterations after which the optimization stops if no improvement
                            surrogate_model ='GP', # surrogate model for the optimization: gaussian process
                            save_models = True,
                            topk = 20)

#save the results of th optimization in a csv file
optResult.save_to_csv("results.csv")

end = time.time()
duration = end - start

print('Optimizing model took: ' + str(round(duration)) + ' seconds.')
results = json.load(open("results/test_LDA/result.json",'r'))

In [ ]:
plt.xlabel('Iteration')
plt.ylabel('Coherence score (c_v)')
plt.title('Mean coherence score per iteration')

coherences = results['dict_model_runs']['Coherence']

mean_coherences = [np.mean(coherences[key]) for key in coherences.keys()]

plt.plot(mean_coherences)
plt.xticks(range(len(mean_coherences)))
plt.show()

print(np.max(mean_coherences))

max_index = np.argmax(mean_coherences)

In [ ]:
print(results["x_iters"].keys())

num_topics = results["x_iters"]["num_topics"][max_index]]

print("Optimal number of topics: ", num_topics)

Now we're ready to train it. Note that the output of a topic model comes as a dictionary composed of 4 elements:


*   *topics*: the list of word topics
*   *topic-word-matrix*: the distribution of the words of the vocabulary for each topic (dimensions: |num topics| x |vocabulary|)
*   *topic-document-matrix*: the distribution of the topics for each document of the training set (dimensions: |num topics| x |training documents|)
*   *test-document-topic-matrix*: the distribution of the topics for each document of the testing set (dimensions: |num topics| x |test documents|)



In [ ]:
model = LDA(num_topics = num_topics, alpha = 'auto', passes= 50, iterations = 200, update_every = 1, chunksize = 1000)

In [ ]:
coherence_list, topic_diversity_list, outputs = [], [], []

for i in tqdm(range(NUM_ITERS)):
  output = model.train_model(dataset, top_words=10)
  outputs.append(output)
  coherence_list.append(ch_score := ch.score(output))
  topic_diversity_list.append(td_score := td.score(output))


print("Mean coherence: ", np.mean(coherence_list), "\nMean topic diversity: ", np.mean(topic_diversity_list))

In [ ]:

# Plot of the coherences of the models (no opt vs opt) and the topic diversity of the models (no opt vs opt). Both plots also show the mean values and the variance!
# Two plots side by side 

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

# Coherence scores plot
axs[0].plot(coherence_list, label="Optimization", color = 'orange', alpha = 0.4)
axs[0].plot(coherence_list_no_opt, label="No optimization", color = 'blue', alpha = 0.4)
axs[0].axhline(y=np.mean(coherence_list), color='orange', linestyle=':')
axs[0].axhline(y=np.mean(coherence_list_no_opt), color='blue', linestyle=':')
axs[0].set_title("Coherence scores")
axs[0].set_xlabel("Training instances")
axs[0].set_ylabel("Coherence score c_v")
axs[0].legend()

# Topic diversity scores plot
axs[1].plot(topic_diversity_list, label="Optimization", color = 'r', alpha = 0.4)
axs[1].plot(topic_diversity_list_no_opt, label="No optimization", color = 'g', alpha = 0.4)
axs[1].axhline(y=np.mean(topic_diversity_list), color='r', linestyle=':')
axs[1].axhline(y=np.mean(topic_diversity_list_no_opt), color='g', linestyle=':')
axs[1].set_title("Topic diversity scores")
axs[1].set_xlabel("Traning instances")
axs[1].set_ylabel("Topic diversity score")
axs[1].legend()

plt.show()


In [ ]:
# Extract the best output from the outputs list according to a linear combination of the coherence and topic diversity scores

best_output = outputs[np.argmax([0.7*coherence_list[i] + 0.3*topic_diversity_list[i] for i in range(len(coherence_list))])]


In [ ]:
topics = best_output['topics']
for i, topic in enumerate(topics):
  wordcloud = WordCloud(width=500, height=400, background_color='white').generate(" ".join(topic))
  plt.figure(figsize=(10, 5))
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis('off')
  plt.title(f"Topic {i}")
  plt.show()


In [ ]:
word_frequencies_barplot_dict(best_output)


In [ ]:
a = best_output['topic-word-matrix']

print(np.shape(a))

print(np.argmax(a[0]))